In [1]:
import pandas as pd
df_train=pd.read_csv('./data/train.txt',sep='\t',names=['label','text'])
df_train.head()

,label,text
0,entertainment,台媒 预测 周 冬雨 金马奖 封后 ， 大气 的 倪妮 却 佳作 难 出
1,food,农村 就是 好 ， 能 吃 到 纯天然 无 添加 的 野生 蜂蜜 ， 营养 又 健康
2,fashion,14 款 知性美 装 ， 时尚 惊艳 搁浅 的 阳光 轻熟 的 优雅
3,history,火焰喷射器 1000 度 火焰 烧死 鬼子 4 连 拍
4,society,18 岁 青年 砍死 88 岁 老兵


In [2]:
df_dev=pd.read_csv('./data/dev.txt',sep='\t',names=['label','text'])
df_dev.head()

,label,text
0,history,跟 日本 刀 、 大马士革 剑 、 阿拉伯 弯刀 比 ， 中国 冷兵器 凭 啥 脱颖而出 ？
1,finance,世界 物 联网 大会 明日 在 京 召开 龙头股 启动 在 即
2,essay,短短 4 句 话 ， 度 了 无数 人
3,sports,郭晶晶 曾 撮合 吴敏霞 与 章子怡 前男友 ， 拒绝 豪门 平淡 才 是 真
4,entertainment,岁月 这 把 杀猪刀 怎么 就 对 周迅 格外 凶残 呢 ?


In [3]:
df_test=pd.read_csv('./data/test_with_label.word',sep='\t',names=['label','text'])
df_test.head()

,label,text
0,baby,生完 小孩 ， 公公 伺候 我 坐月子 ， 很 羞涩 很 感动
1,fashion,唐艺昕 与 陈伟霆 为 初秋 的 情侣 做出 了 典范 ， 看 他们 如何 穿 情侣 衣
2,entertainment,同学聚会 美女 被 嘲笑 是 剩女 ， 当超帅 老公 带 着 儿子 出场 ， 全场 沸腾 了
3,finance,中国 供给 侧 至少 存在 六大 问题
4,world,2.5 万英镑 可住 戴妃 闺房


In [4]:
#train_X=df_train.sample(frac=1).reset_index(drop=True)
train_X=df_train
train_Y=train_X['label']
train_X=train_X['text']
#dev_X=df_dev.sample(frac=1).reset_index(drop=True)
dev_X=df_dev
dev_Y=dev_X['label']
dev_X=dev_X['text']
#test_X=df_test.sample(frac=1).reset_index(drop=True)
test_X=df_test
test_Y=test_X['label']
test_X=test_X['text']

In [5]:
def find_Chinese(text): #去除非汉字
    words=text.split(' ')
    Chinese_txt=''
    for word in words:
        if word>=u'\u4e00' and word<=u'\u9fa5':
            Chinese_txt+=word+' '
    return Chinese_txt

In [6]:
file1='./data/中文停用词表.txt'
file2='./data/哈工大停用词表.txt'
file3='./data/四川大学机器智能实验室停用词库.txt'
stopwords_set=set()
with open(file1,'r',encoding='utf-8') as f1,\
    open(file2,'r',encoding='utf-8') as f2,\
    open(file3,'r',encoding='utf-8') as f3:
    for word in f1.readlines():
        stopwords_set.add(word.strip())
    for word in f2.readlines():
        stopwords_set.add(word.strip())
    for word in f3.readlines():
        stopwords_set.add(word.strip())

def remove_stopwords(text): #去除停用词
    words=text.split(' ')
    Chinese_txt=''
    for word in words:
        if word not in stopwords_set:
            Chinese_txt+=word+' '
    return Chinese_txt

In [7]:
def clean(df): #文本清洗
    for i in range(len(df)):
        df[i]=find_Chinese(df[i])
        df[i]=remove_stopwords(df[i])
    return df

In [8]:
train_X=clean(train_X)
dev_X=clean(dev_X)
test_X=clean(test_X)

In [9]:
def remove_empty(df_X,df_Y):
    remove_list=[]
    for i in range(len(df_X)):
        if df_X[i]=='':
            remove_list.append(i)
    remove_list.reverse()
    for i in remove_list:
        del df_X[i]
        del df_Y[i]
    return df_X,df_Y

train_X,train_Y=remove_empty(train_X,train_Y)
dev_X,dev_Y=remove_empty(dev_X,dev_Y)
test_X,test_Y=remove_empty(test_X,test_Y)

In [10]:
print(train_X.shape)
print(dev_X.shape)
print(test_X.shape)

(155746,)
(35011,)
(35968,)


In [11]:
import os
if not os.path.exists(os.getcwd()+'\\model'):
    os.mkdir(os.getcwd()+'\\model')
    os.chdir(os.getcwd()+'\\model')

In [12]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer()
transformer=TfidfTransformer()
corpus=train_X.tolist()+dev_X.tolist()+test_X.tolist()
tfidf=transformer.fit_transform(vectorizer.fit_transform(corpus))
import joblib
joblib.dump(transformer,'TfidfTransformer.pkl')
joblib.dump(vectorizer,'CountVectorizer.pkl')
words=vectorizer.get_feature_names_out()
print('how many words: {0}'.format(len(words)))
print('tf-idf shape: ({0},{1})'.format(tfidf.shape[0],tfidf.shape[1]))

how many words: 135901
tf-idf shape: (226725,135901)


In [13]:
from sklearn import preprocessing
encoder=preprocessing.LabelEncoder()
corpus_label=train_Y.tolist()+dev_Y.tolist()+test_Y.tolist()
corpus_encode_label=encoder.fit_transform(corpus_label)
train_label=corpus_encode_label[:train_X.shape[0]]
dev_label=corpus_encode_label[train_X.shape[0]:train_X.shape[0]+dev_X.shape[0]]
test_label=corpus_encode_label[train_X.shape[0]+dev_X.shape[0]:]
train_set=tfidf[:train_X.shape[0]]
dev_set=tfidf[train_X.shape[0]:train_X.shape[0]+dev_X.shape[0]]
test_set=tfidf[train_X.shape[0]+dev_X.shape[0]:]

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

lr_model=LogisticRegression()
lr_model.fit(train_set,train_label)
print('val mean accuracy: {0}'.format(lr_model.score(dev_set,dev_label)))
y_pred=lr_model.predict(test_set)
print(classification_report(test_label,y_pred))

val mean accuracy: 0.7397389391905401
              precision    recall  f1-score   support

           0       0.81      0.87      0.84      1998
           1       0.85      0.81      0.83      1997
           2       0.94      0.74      0.83      2000
           3       0.62      0.74      0.68      1999
           4       0.86      0.57      0.68      1994
           5       0.73      0.78      0.75      2000
           6       0.75      0.76      0.75      1998
           7       0.73      0.82      0.78      1995
           8       0.82      0.80      0.81      1999
           9       0.73      0.80      0.76      2000
          10       0.78      0.80      0.79      2000
          11       0.87      0.63      0.73      1999
          12       0.42      0.62      0.50      1998
          13       0.82      0.83      0.82      1999
          14       0.88      0.55      0.67      1999
          15       0.74      0.75      0.74      1997
          16       0.63      0.77      0.69

c:\Users\DCrusher\.conda\envs\Graduation\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [15]:
from sklearn.ensemble import RandomForestClassifier
rf_model=RandomForestClassifier()
rf_model.fit(train_set,train_label)
print('val mean accuracy: {0}'.format(rf_model.score(dev_set,dev_label)))
y_pred=rf_model.predict(test_set)
print(classification_report(test_label,y_pred))

val mean accuracy: 0.6817285995829881
              precision    recall  f1-score   support

           0       0.73      0.87      0.79      1998
           1       0.78      0.76      0.77      1997
           2       0.86      0.77      0.81      2000
           3       0.64      0.60      0.62      1999
           4       0.52      0.75      0.61      1994
           5       0.63      0.71      0.67      2000
           6       0.68      0.70      0.69      1998
           7       0.65      0.75      0.70      1995
           8       0.75      0.74      0.74      1999
           9       0.72      0.69      0.70      2000
          10       0.74      0.72      0.73      2000
          11       0.77      0.58      0.66      1999
          12       0.46      0.45      0.45      1998
          13       0.82      0.72      0.77      1999
          14       0.82      0.49      0.61      1999
          15       0.67      0.68      0.67      1997
          16       0.58      0.66      0.62

In [16]:
import joblib
joblib.dump(lr_model,'LogisticRegression.pkl')
joblib.dump(rf_model,'RandomForest.pkl')

['RandomForest.pkl']

In [17]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
kn_model=KNeighborsClassifier()
kn_model.fit(train_set,train_label)
print('val mean accuracy: {0}'.format(kn_model.score(dev_set,dev_label)))
y_pred=kn_model.predict(test_set)
print(classification_report(test_label,y_pred))

val mean accuracy: 0.12019079717802976
              precision    recall  f1-score   support

           0       0.25      0.22      0.23      1998
           1       0.14      0.14      0.14      1997
           2       0.91      0.12      0.21      2000
           3       0.58      0.03      0.06      1999
           4       0.06      0.81      0.12      1994
           5       0.13      0.11      0.12      2000
           6       0.30      0.09      0.14      1998
           7       0.17      0.16      0.17      1995
           8       0.62      0.07      0.12      1999
           9       0.80      0.03      0.07      2000
          10       0.78      0.02      0.04      2000
          11       0.43      0.07      0.12      1999
          12       0.15      0.03      0.05      1998
          13       0.63      0.01      0.02      1999
          14       0.55      0.08      0.14      1999
          15       0.22      0.07      0.11      1997
          16       0.78      0.07      0.1

In [18]:
dc_model=DecisionTreeClassifier()
dc_model.fit(train_set,train_label)
print('val mean accuracy: {0}'.format(dc_model.score(dev_set,dev_label)))
y_pred=dc_model.predict(test_set)
print(classification_report(test_label,y_pred))

val mean accuracy: 0.5944131844277513
              precision    recall  f1-score   support

           0       0.70      0.79      0.74      1998
           1       0.72      0.68      0.70      1997
           2       0.81      0.67      0.74      2000
           3       0.51      0.51      0.51      1999
           4       0.47      0.65      0.54      1994
           5       0.56      0.61      0.59      2000
           6       0.61      0.62      0.61      1998
           7       0.61      0.66      0.63      1995
           8       0.68      0.66      0.67      1999
           9       0.61      0.62      0.62      2000
          10       0.64      0.64      0.64      2000
          11       0.68      0.49      0.57      1999
          12       0.37      0.39      0.38      1998
          13       0.71      0.65      0.68      1999
          14       0.72      0.46      0.56      1999
          15       0.59      0.61      0.60      1997
          16       0.51      0.55      0.53

In [19]:
joblib.dump(dc_model,'DecisionTree.pkl')
joblib.dump(kn_model,'KNeighborsClassifier.pkl')

['KNeighborsClassifier.pkl']

In [20]:
ab_model=AdaBoostClassifier()
ab_model.fit(train_set,train_label)
print('val mean accuracy: {0}'.format(ab_model.score(dev_set,dev_label)))
y_pred=ab_model.predict(test_set)
print(classification_report(test_label,y_pred))

val mean accuracy: 0.22787124046728172
              precision    recall  f1-score   support

           0       0.79      0.62      0.69      1998
           1       0.79      0.06      0.10      1997
           2       0.77      0.52      0.62      2000
           3       0.07      0.93      0.13      1999
           4       0.88      0.13      0.23      1994
           5       0.83      0.07      0.13      2000
           6       0.00      0.00      0.00      1998
           7       0.94      0.28      0.43      1995
           8       0.89      0.32      0.47      1999
           9       0.76      0.15      0.26      2000
          10       0.43      0.36      0.39      2000
          11       0.00      0.00      0.00      1999
          12       0.00      0.00      0.00      1998
          13       0.00      0.00      0.00      1999
          14       0.83      0.27      0.40      1999
          15       0.80      0.17      0.29      1997
          16       0.78      0.05      0.0

c:\Users\DCrusher\.conda\envs\Graduation\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\DCrusher\.conda\envs\Graduation\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\DCrusher\.conda\envs\Graduation\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
joblib.dump(ab_model,'AdaBoost.pkl')

['AdaBoost.pkl']